In [122]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import requests
import re
from bs4 import BeautifulSoup
import time 
from tqdm import tqdm
import random
import pandas as pd

In [2]:
def get_sitemap_urls(sitemap_url):
    Pro=[]
    for i in sitemap_url:
        response = requests.get(i)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            urls = [loc.text for loc in soup.find_all('loc')]
            Pro.extend(urls) 
        else:
            print(f"Failed to fetch sitemap. Status Code: {response.status_code}")
    return Pro     

In [3]:
def unique(Orignal_list):
    unique_list = [item for index, item in enumerate(Orignal_list) if item not in Orignal_list[:index]]
    return unique_list

In [4]:
def scrollWindow():
    time.sleep(2)
    initial_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        driver.implicitly_wait(2) 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == initial_height:
            break
        initial_height = new_height
       

In [301]:
def get_product_info(URL):
    driver.get(URL)
    scrollWindow()
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    ProductName = soup.find('h1').text
    df = []
    if soup.find_all('h2')[0].text not in ['This item is out of stock.', 'Item no longer available.']:
        try:
            Rating = soup.find('span', class_="tt-c-reviews-summary__rating-number").text
            ReviewCount = soup.find('div', class_="tt-c-reviews-summary__heading tt-u-mb--xs").text
        except:
            Rating = '0'
            ReviewCount = 'No Ratings'
        try:
            for i in range(len(soup.find('datalist').find_all('a'))-1):
                SubProduct = soup.find('datalist').find_all('a')[i].find('span', class_='item-name').text
                try:
                    RegularPriceElement = soup.find('datalist').find_all('a')[i].find('span', class_='item-price').find('span', class_='item-regular-price')
                    RegularPrice = RegularPriceElement.text.replace('Each','') if RegularPriceElement else None
                    SalePriceElement = soup.find('datalist').find_all('a')[i].find('span', class_='item-price').find('span', {'aria-label': re.compile(r'Sale price \$\d+\.\d+')})
                    SalePrice = SalePriceElement.text.replace('Each','')
                except:
                    RegularPrice= None
                    SalePrice = soup.find('datalist').find_all('a')[i].find('span', class_='item-price').text.replace('Each','')
                if SalePrice != '' :
                    row = {
                    'ProductURL':URL,
                    'ProductName':ProductName,
                    'SubProduct': SubProduct,
                    'RegularPrice':RegularPrice,
                    'SalePrice':SalePrice,
                    'ReviewCount': ReviewCount,
                    'Rating':Rating}
                    df.append(row)
        except Exception as e:
            SubProduct=None
            try:
                RegularPriceElement = soup.find('datalist').find_all('a')[i].find('span', class_='item-price').find('span', class_='item-regular-price')
                RegularPrice = RegularPriceElement.text.replace('Each','') if RegularPriceElement else None
                SalePriceElement = soup.find('datalist').find_all('a')[i].find('span', class_='item-price').find('span', {'aria-label': re.compile(r'Sale price \$\d+\.\d+')})
                SalePrice = SalePriceElement.text.replace('Each','')
            except:
                RegularPrice= None
                SalePrice = soup.find('div', class_="no-sale-price").text.replace('Each','')
            if SalePrice != '' :
                row = {
                    'ProductURL':URL,
                    'ProductName':ProductName,
                    'SubProduct': SubProduct,
                    'RegularPrice':RegularPrice,
                    'SalePrice':SalePrice,
                    'ReviewCount': ReviewCount,
                    'Rating':Rating
                }
                df.append(row)
    return df

In [302]:
allowed_user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/14.1.2",
    "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"   
]

In [11]:
sitemap_url = ['https://www.starkbros.com/datafeeds/sitemap.xml']
sitemap_urls = get_sitemap_urls(sitemap_url)
Pro_URL= []
for i in sitemap_urls:
    if 'https://www.starkbros.com/products/' in i and len(i.split('/'))>=7:
        Pro_URL.append(i)
df = pd.DataFrame(Pro_URL)
df.to_csv('Links.csv',index=False)

In [303]:
Pro_URL.index('https://www.starkbros.com/products/fruit-trees/apple-trees/pink-lady-apple')
Pro_URL[959+15+929]

'https://www.starkbros.com/products/garden-plants/additional-garden-plants/red-fanal-astilbe'

In [286]:
Product_details=[]

In [294]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[1903:], desc="GettingData", unit="URL"):
    df = get_product_info(i)
    Product_details.extend(df)
    #print(df)
daf = pd.DataFrame(Product_details)
daf.to_csv('StarkBros.csv',index=False)
driver.quit()
Product_details

GettingData: 100%|██████████████████████████████████████████████████████████████████| 183/183 [14:54<00:00,  4.89s/URL]


[{'ProductURL': 'https://www.starkbros.com/products/nut-trees/almond-trees/aldrich-almond',
  'ProductName': 'Aldrich Almond Tree',
  'SubProduct': 'Supreme, EZ Start®',
  'RegularPrice': None,
  'SalePrice': '$86.99Each',
  'ReviewCount': 'No Ratings',
  'Rating': '0'},
 {'ProductURL': 'https://www.starkbros.com/products/nut-trees/almond-trees/aldrich-almond',
  'ProductName': 'Aldrich Almond Tree',
  'SubProduct': 'Supreme XL, 9x9x12" Pot',
  'RegularPrice': None,
  'SalePrice': '$118.99Each',
  'ReviewCount': 'No Ratings',
  'Rating': '0'},
 {'ProductURL': 'https://www.starkbros.com/products/nut-trees/almond-trees/all-in-one-almond',
  'ProductName': 'All-In-One Almond Tree',
  'SubProduct': None,
  'RegularPrice': None,
  'SalePrice': '$59.99',
  'ReviewCount': '88 Ratings',
  'Rating': '3.3'},
 {'ProductURL': 'https://www.starkbros.com/products/nut-trees/almond-trees/bonnie-almond',
  'ProductName': 'Bonnie Almond Tree',
  'SubProduct': 'Dwarf, EZ Start®',
  'RegularPrice': None,
